## 演習 : クレジットカードの支払履歴とデフォルトのデータを使ったモデリング

datasetフォルダにある"UCI_Credit_Card.csv"は2005年4月~9月の顧客別のクレジットカードの支払履歴及び翌月にデフォルトしたかどうかのデータセットです。
本データは、https://www.kaggle.com/uciml/default-of-credit-card-clients-dataset から取得しました。

### 課題
あなたはカード会社のデータサイエンティストです。
現在デフォルト率が約22%となっており、デフォルト率を下げるとともに、機会損失も返済不能に陥る実際の損失も最小化するモデルを作るように依頼されました。
「今回は中身はBlackboxで良いから、とにかく当ててくれ！」という依頼がきています。
あなたはどのようなモデルを作りますか？

### 注意
* データセットは以下の2つに分割しています
    * train_UCI_Credit_Card.csv
    * test_UCI_Credit_Card.csv
* モデリングはtrain_UCI_Credit_Card.csvをのみを使ってください。
* test_UCI_Credit_Card.csvを答え合わせに使います。
* 私が作成したevaluate_test_data関数に以下を指定してください
    * 学習済みのモデル
    * 推定デフォルト確率をいくつ以上であればローンを出さないこととするかを決めるcut_off
    * テストデータ

### ヒント
* 特徴量を頑張って作る
* アルゴリズムやモデル選択の際に、

>### Dataset Information

>This dataset contains information on default payments, demographic factors, credit data, history of payment, and bill statements of credit card clients in Taiwan from April 2005 to September 2005.

>### Content

>There are 25 variables:

>ID: ID of each client

>LIMIT_BAL: Amount of given credit in NT dollars (includes individual and family/supplementary credit

>SEX: Gender (1=male, 2=female)

>EDUCATION: (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown)

>MARRIAGE: Marital status (1=married, 2=single, 3=others)

>AGE: Age in years

>PAY_0: Repayment status in September, 2005 (-1=pay duly, 1=payment delay for one month, 2=payment delay for two months, ... 8=payment delay for eight months, 9=payment delay for nine months and above)

>PAY_2: Repayment status in August, 2005 (scale same as above)

>PAY_3: Repayment status in July, 2005 (scale same as above)

>PAY_4: Repayment status in June, 2005 (scale same as above)

>PAY_5: Repayment status in May, 2005 (scale same as above)

>PAY_6: Repayment status in April, 2005 (scale same as above)

>BILL_AMT1: Amount of bill statement in September, 2005 (NT dollar)

>BILL_AMT2: Amount of bill statement in August, 2005 (NT dollar)

>BILL_AMT3: Amount of bill statement in July, 2005 (NT dollar)

>BILL_AMT4: Amount of bill statement in June, 2005 (NT dollar)

>BILL_AMT5: Amount of bill statement in May, 2005 (NT dollar)

>BILL_AMT6: Amount of bill statement in April, 2005 (NT dollar)

>PAY_AMT1: Amount of previous payment in September, 2005 (NT dollar)

>PAY_AMT2: Amount of previous payment in August, 2005 (NT dollar)

>PAY_AMT3: Amount of previous payment in July, 2005 (NT dollar)

>PAY_AMT4: Amount of previous payment in June, 2005 (NT dollar)

>PAY_AMT5: Amount of previous payment in May, 2005 (NT dollar)

>PAY_AMT6: Amount of previous payment in April, 2005 (NT dollar)

>default.payment.next.month: Default payment (1=yes, 0=no)

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

try:
    xrange
except NameError:
    xrange = range

from __future__ import print_function

## データの読み込み

まず、データを読み込みましょう。

In [ ]:
train_data = pd.read_csv("data/train_UCI_Credit_Card.csv")

In [ ]:
train_data.head()

## 列名の確認

どんな列があったかを思い出しましょう。PandasのDataFrameは、DataFrame.columnsとやると、列名を確認できます。

ここでは、見やすいように、forループを回しています。

In [ ]:
for colname in train_data.columns:
    print(colname)

## 特徴量の設計（Feature Engneering)
みなさんの想像を膨らませて、新たな変数を作ってください。

例: BILL_AMTを全部足しあげて、新しい列：BILL_AMT_TOTAL　を作る。

In [ ]:
train_data["BILL_AMT_TOTAL"] = train_data["BILL_AMT1"] + train_data["BILL_AMT2"] + train_data["BILL_AMT3"]  + train_data["BILL_AMT4"] + train_data["BILL_AMT5"] + train_data["BILL_AMT6"]

In [ ]:
train_data.head()

## developmentデータを作っておきましょう。

ここでは、学習に使うデータとチューニング用のデータとを 80:20に分けます。

In [ ]:
from sklearn.cross_validation import train_test_split

In [ ]:
train_data2, dev_data = train_test_split(train_data, test_size = 0.2, random_state=1234)

In [ ]:
print("Number of Rows: ", train_data2.shape[0])
train_data2.head()

In [ ]:
print("Number of Rows: ", dev_data.shape[0])
dev_data.head()

### 機械学習のアルゴリズムをインポートします。

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, recall_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

#講義ではカバーしていませんが、興味がある方はこちらもどうぞ 
# http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
# from sklearn.ensemble import GradientBoostingClassifier

# http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html
# from sklearn.ensemble import BaggingClassifier

# http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html
# from sklearn.ensemble import AdaBoostClassifier

### 例: ロジスティック回帰の場合

### まず、説明変数と被説明変数を決めます。

In [ ]:
predictor_feature = ["BILL_AMT_TOTAL", "AGE"] #説明変数
target_feature = "default" #被説明変数

### もし特定の列だけ抜きたいのあれば、こんなやり方もあり

In [ ]:
exclude_cols = ["ID", "default"] #除外したい列をこちらに入れておきます。
predictor_feature = []

for colname in train_data.columns:
    #除外したい列に列名が含まれていなければ、True
    if colname not in exclude_cols:
        predictor_feature.append(colname)  #predictor_featureのリストに追加

print(predictor_feature)

In [ ]:
#学習データ
train_X = train_data2[predictor_feature]
train_y = train_data2[target_feature]

#Developmentデータ
dev_X = dev_data[predictor_feature]
dev_y = dev_data[target_feature]

### まず初期化します

In [ ]:
clf = LogisticRegression(C = 10.0)

### 続いて、とりあえず学習させます（チューニングなし）。

In [ ]:
clf.fit(train_X, train_y)

### Developmentデータで試します。

In [ ]:
dev_y_pred_proba  = clf.predict_proba(dev_X)

### 2値分類問題ですので、このように2列で出力されます。

In [ ]:
dev_y_pred_proba

###  チューニングをしてみます

例： Logistic regressionのパラメーターはCです（これはオーバーフィッティングを防ぐための罰則、ペナルティの強さを表します）
ここではF1-measureを目安にチューニングしてみましょう。また、カットオフ値は決め打ちで0.24にしておきます。

In [ ]:
#パラメータの候補を出しておきます。
C_cancidates = np.array([0.01, 0.1, 1.0, 10, 100])

#決め打ちのCut offを指定します。
cut_off = 0.24

#トラックしたい評価指標を入れておく、空っぽのリストを用意します。
f1_measure_history = []

#Forループでパラメータの候補を一つずつ試します。
for each_C in C_cancidates:
    print("----------------------------------------------------------------")
    print("Parameter C = ", each_C)
    
    #初期化
    clf= LogisticRegression(C=each_C)
    
    #学習
    clf.fit(train_X, train_y)
    
    #devデータで予測します（2列のデータで出てくるので、2列目、つまりデフォルトする確率だけを抜き取ります）
    dev_y_pred_proba = clf.predict_proba(dev_X)[:,1]
    
    #貸さないなら、1、貸すなら0にします。
    pred_flag = (dev_y_pred_proba >cut_off ).astype(int)
    
    #混同行列を出力します。
    print(confusion_matrix(y_pred=pred_flag, y_true=dev_y))
    
    #F1scoreを計算します
    each_f1_score = f1_score(y_pred=pred_flag, y_true=dev_y)  #もしF1scoreではなく、precisionやrecallを使いたい場合は変えましょう
    print("F1 score: ", each_f1_score)
    
    #最後に、f1_measure_historyにそれぞれのf1 scoreを追加します。
    f1_measure_history.append(each_f1_score)

#Forループが終わったら、一番よかったf1 scoreを取り出し、対応するパラメータを出力します。
max_score = max(f1_measure_history)
best_param = C_cancidates[np.array(f1_measure_history) == max_score]
print("Max score: ", max_score, "When C =", best_param)

## チューニング後のモデルを学習しておきましょう（後で、テストデータに適用するために）

In [ ]:
mybest_model = LogisticRegression(C=1.0)
mybest_model.fit(train_X, train_y)

## 損失額を計算するための関数: evaluate_test_data

In [ ]:
def evaluate_test_data(mymodel, cut_off, data_df, predictor_feature, target_feature, verbose = True):
    
    # TO DO: 
    #もしtrainデータで新しい列を作成したらtest_dataでも作ってください
    #trainingデータで作成した変数を作っておきます。
    data_df["BILL_AMT_TOTAL"] = data_df["BILL_AMT1"] + data_df["BILL_AMT2"] + data_df["BILL_AMT3"]  + data_df["BILL_AMT4"] + data_df["BILL_AMT5"] + data_df["BILL_AMT6"]
    
    
    X = data_df[predictor_feature]
    y = data_df[target_feature]
    
    #デフォルト確率の予測
    y_pred_proba = mymodel.predict_proba(X)[:,1]
    
    #cut_offより大きければデフォルトする=つまりローンを出さない
    pred_flag = (y_pred_proba >cut_off ).astype(int)
    
    #予測精度の確認(Confusuion Matrix)
    if verbose:
        print(confusion_matrix(y_true=y, y_pred=pred_flag))
        print("Accuracy: ", accuracy_score(y_true=y, y_pred=pred_flag))
        print("Recall: ", recall_score(y_true=y, y_pred=pred_flag))
        print("Precision: ", precision_score(y_true=y, y_pred=pred_flag))
        print("F1 score: ", f1_score(y_true=y, y_pred=pred_flag) )
    
    #テストデータのデフォルト率
    base_default_rate = np.mean(y)
    if verbose:
        print("テストデータのデフォルト率: ", base_default_rate)
    
    #デフォルトしないと予測した中で本当にデフォルトした人の割合
    fail_rate = np.mean(y[pred_flag==0])
    if verbose:
        print("デフォルトしないと予測した中で本当にデフォルトした人の割合: ", fail_rate)
    
    #本当は貸した方がよかったのに、貸さなかったバランス（機会ロス）
    oploss_idx = np.logical_and(y==0, pred_flag==1)
    opportunity_loss = sum(data_df["LIMIT_BAL"][oploss_idx])
    if verbose:
        print("本当は貸した方がよかったのに、貸さなかったバランス（機会ロス）: " , opportunity_loss)
    
    #本当は貸さない方がよかったのに、貸してしまったバランス（返済不能額）
    realloss_idx = np.logical_and(y==1, pred_flag==0)
    realized_loss = sum(data_df["LIMIT_BAL"][realloss_idx])
    if verbose:
        print("本当は貸さない方がよかったのに、貸してしまったバランス（返済不能額）: " , realized_loss)
    
    #Total loss
    
    total_loss = opportunity_loss + realized_loss
    if verbose:
        print("Total Loss: ", total_loss)
    
    return total_loss

## devデータを使って、cut_off値を最適化する

カットオフ値の候補を作る(numpyのarangeで作れます）

In [ ]:
cut_off_candidates = np.arange(0, 1, 0.01)

一つずつ試しましょう

In [ ]:
total_loss_history = []
for each_cut_off in cut_off_candidates:
    each_total_loss = evaluate_test_data(mymodel=mybest_model, 
                                         cut_off=each_cut_off, 
                                         data_df=dev_data, 
                                         predictor_feature=predictor_feature, 
                                        target_feature = target_feature, 
                                        verbose = False)
    
    total_loss_history.append(each_total_loss)

### 最小のTotal Lossを探す

In [ ]:
 #リストだとやりにくいので、Numpyの配列にしておきます
cut_off_candidates_ar = np.array(cut_off_candidates)
total_loss_history_ar = np.array(total_loss_history)

#グラフにします
plt.plot(cut_off_candidates,  total_loss_history)
best_cut_off  = cut_off_candidates_ar[total_loss_history_ar == min(total_loss_history_ar)]
print("Total Loss: ", min(total_loss_history), "When cut off = ", best_cut_off)

## テストデータで試してみよう（これは演習の最後の1回だけ！）

テストデータを読み込みます

In [ ]:
test_data = pd.read_csv("data/test_UCI_Credit_Card.csv")

いよいよ、テストデータで検証してみます（evaluate_test_dataで、特長量の設計をしていたら、関数内に書いておかないだめですよ）

ドキドキしますね！

In [ ]:
evaluate_test_data(mymodel=mybest_model, 
                   cut_off = 0.2, 
                   data_df = test_data, 
                   predictor_feature = predictor_feature, 
                   target_feature = target_feature,
                   verbose = True
                  )